In [ ]:
import numpy as np
import png
import pydicom
from sklearn.preprocessing import normalize

from os import listdir
from os.path import isfile, join
import os
import torch
import torch.nn as nn

from roi import RoiLearn
from preprocessor import Preprocessor

In [1]:
import dicom_reader
import con_reader
from con2img import draw_square
from matplotlib.pyplot import plot, imshow, show,imsave
import csv
import os
import glob

def write_all_rectangle2file(rootdir):
    rectangle_file = rootdir + 'rectangle.csv'
    for subdir, dirs, files in os.walk(rootdir):
        dcm_folder = glob.glob(subdir + './*.dcm')
        if len(dcm_folder) != 0 and len(glob.glob(subdir.rsplit('\\', 1)[0] + '/*.con')) != 0:
            dcm_folder = subdir 
            con_file = glob.glob(dcm_folder.rsplit('\\', 1)[0] + '/*.con')[0]
            print(dcm_folder+ '\\', con_file, rectangle_file)
            write_rectangle2file(dcm_folder+ '\\', con_file, rectangle_file)
        

#Write embracing rectangle info into a rectang.csv file
#dcm_folder - location where your .dcm images take place
#con_file - file which associated with the above .dcm images
#rectangle_file - the location where you want to place the rectangle info file
def write_rectangle2file(dcm_folder, con_file, rectangle_file):

    dc = dicom_reader.DCMreader(dcm_folder)
    print("Dicom files were read in!")
    cn = con_reader.CONreader(con_file)
    print("Con files were read in!")

    hierarchical = cn.get_hierarchical_contours()

    if os.path.exists(rectangle_file):
        os.remove(rectangle_file)
        
    for slice in hierarchical.keys():
        for frame in hierarchical[slice].keys():

            img = dc.get_image(slice, frame)
            xglobalmin = 20000
            xglobalmax = 0
            yglobalmin = 20000
            yglobalmax = 0
            for mode in hierarchical[slice][frame].keys():
                # find min-max for rectangle
                
                for contour in hierarchical[slice][frame][mode]:
                    x_vec = contour['x']
                    y_vec = contour['y']

                    min_x,min_y = 20000, 20000
                    max_x,max_y = 0 , 0
                    #find local min-max
                    for idx in range(len(x_vec)):

                        if x_vec[idx] > max_x:
                            max_x = x_vec[idx]
                        if min_x > x_vec[idx]:
                            min_x = x_vec[idx]

                        if y_vec[idx] > max_y:
                            max_y = y_vec[idx]
                        if min_y > y_vec[idx]:
                            min_y = y_vec[idx]
                            
                #comparism with global min-max
                if max_x > xglobalmax:
                    xglobalmax = max_x
                if xglobalmin > min_x:
                    xglobalmin = min_x
                if max_y > yglobalmax:
                    yglobalmax = max_y
                if yglobalmin > min_y:
                    yglobalmin = min_y

            for x_coord in range(int(xglobalmax - xglobalmin)):
                for y_coord in range(int(yglobalmax - yglobalmin)):
                    draw_square(img, int(y_coord + yglobalmin), int(x_coord + xglobalmin))
            
            with open(rectangle_file, 'a',newline='') as myfile:
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL, delimiter=';')
                #slice, frame number, x, y, height, widht
                wr.writerow([slice, frame, xglobalmin, yglobalmin, int(yglobalmax - yglobalmin), int(xglobalmax - xglobalmin)])



#Draw embracing rectangle around contours.
#dcm_folder - location where your .dcm images take place
#con_file - file which associated with the above .dcm images
def draw_contour_rectangle2images(dcm_folder, con_file):

    dc = dicom_reader.DCMreader(dcm_folder)
    print("Dicom files were read in!")
    cn = con_reader.CONreader(con_file)
    print("Con files were read in!")

    hierarchical = cn.get_hierarchical_contours()

    for slice in hierarchical.keys():
        for frame in hierarchical[slice].keys():

            img = dc.get_image(slice, frame)
            xglobalmin = 20000
            xglobalmax = 0
            yglobalmin = 20000
            yglobalmax = 0
            for mode in hierarchical[slice][frame].keys():
                # find min-max for rectangle
                
                for contour in hierarchical[slice][frame][mode]:
                    x_vec = contour['x']
                    y_vec = contour['y']

                    min_x,min_y = 20000, 20000
                    max_x,max_y = 0 , 0
                    #find local min-max
                    for idx in range(len(x_vec)):

                        if x_vec[idx] > max_x:
                            max_x = x_vec[idx]
                        if min_x > x_vec[idx]:
                            min_x = x_vec[idx]

                        if y_vec[idx] > max_y:
                            max_y = y_vec[idx]
                        if min_y > y_vec[idx]:
                            min_y = y_vec[idx]
                            
                #comparism with global min-max
                if max_x > xglobalmax:
                    xglobalmax = max_x
                if xglobalmin > min_x:
                    xglobalmin = min_x
                if max_y > yglobalmax:
                    yglobalmax = max_y
                if yglobalmin > min_y:
                    yglobalmin = min_y

            for x_coord in range(int(xglobalmax - xglobalmin)):
                for y_coord in range(int(yglobalmax - yglobalmin)):
                    draw_square(img, int(y_coord + yglobalmin), int(x_coord + xglobalmin))
                
                
            imsave('imgs/img_' + str(slice) + '_' + str(frame) + '.png', img)


In [4]:
write_all_rectangle2file('O:\\ProgrammingSoftwares\\anaconda_projects\\heart_contour\\sa_all_1\\')

O:\ProgrammingSoftwares\anaconda_projects\heart_contour\sa_all_1\17108976AMR804\1001\imgs\ O:\ProgrammingSoftwares\anaconda_projects\heart_contour\sa_all_1\17108976AMR804\1001\contour.con O:\ProgrammingSoftwares\anaconda_projects\heart_contour\sa_all_1\rectangle.csv
Dicom files were read in!
Con files were read in!
O:\ProgrammingSoftwares\anaconda_projects\heart_contour\sa_all_1\17128355AMR812\1001\imgs\ O:\ProgrammingSoftwares\anaconda_projects\heart_contour\sa_all_1\17128355AMR812\1001\contour.con O:\ProgrammingSoftwares\anaconda_projects\heart_contour\sa_all_1\rectangle.csv
Dicom files were read in!


KeyboardInterrupt: 

In [ ]:
roii = RoiLearn()
preprocessor = Preprocessor()

dataLoader = preprocessor.create_datasetLoaderDCM('./root')
preprocessor.show_some_loadedData()

roii.build_model()

roii.propagate_from_dataLoader(dataLoader)
#y_pred = roi.propagate().detach().numpy()
#save_image(y_pred[0,0,:,:]*255, 'pred.png')

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from dicom_reader import DCMreader

class RoiDataset(Dataset):
    """Roi dataset."""

    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with contour data.
            root_dir (string): Directory for .dcm images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        'height is  (y length) width is  (x length)'
        self.contour_data = pd.read_csv(csv_file,sep=';', names=('slice', 'frame', 'xmin', 'ymin', 'height','width' ))
        self.dcm_images = DCMreader(root_dir)
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.contour_data)
    
    def __getitem__(self, idx):
        cont = ds.contour_data.iloc[idx]
        sl = int(cont['slice'])
        fr = int(cont['frame'])
        
        mask = np.zeros((224,224))
        mask[ int(cont['ymin']): int(cont['ymin']) + int(cont['height']),int(cont['xmin']): int(cont['xmin']) + int(cont['width'])] = 1
        image = self.dcm_images.get_image(sl,fr)
        
        sample = {'image': image, 'mask': mask}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
        if self.target_transform:
            sample['mask'] = self.target_transform(sample['mask'])

        return sample
        

In [ ]:
from sklearn.preprocessing import StandardScaler
from skimage.transform import resize


class ReScale64(object):
    """Scale down ndarrays in sample to Tensors."""

    def __call__(self, data):
        data = resize(data, (64, 64))
        return data


class ReScale32(object):
    """Scale down ndarrays in sample to Tensors."""

    def __call__(self, data):
        data = resize(data, (32, 32))
        data = np.reshape(data, (1024))
        return data
    

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, data):
        return torch.from_numpy(data)
    
class StandardScale(object):
    """Standard scale ndarrays."""

    def __call__(self, data):
        scaler = StandardScaler()
        data = scaler.fit_transform(data)
        
        # add color axis because
        # numpy image: H x W
        # torch image: C X H X W
        data = np.expand_dims(data, axis=0)
        return data


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision.transforms import Compose
from roi_dataset import RoiDataset
import transform_classes
from roi import RoiLearn
import torch

dcm_folder = "O:\\ProgrammingSoftwares\\anaconda_projects\\heart_contour\\sa_all_1\\17108976AMR804\\1001\\imgs\\"
con_file = "O:\\ProgrammingSoftwares\\anaconda_projects\\heart_contour\\sa_all_1\\17108976AMR804\\1001\\contour.con"
csv_file = 'O:/ProgrammingSoftwares/anaconda_projects/heart_contour/sa_all_1/rectangle.csv'


write_rectangle2file(dcm_folder, con_file, csv_file)


compose1 = Compose([transform_classes.ReScale64(),transform_classes.StandardScale(),transform_classes.ToTensor()])
compose2 = Compose([transform_classes.ReScale32(),transform_classes.ToTensor()])
ds = RoiDataset(csv_file, dcm_folder, compose1, compose2)

roi = RoiLearn()
roi.build_model()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(roi.model.parameters(), lr=0.001)

dataset_loader = torch.utils.data.DataLoader(ds,batch_size=8, shuffle=True,num_workers=0)

In [ ]:
learning_changed = False
for epoch in range(100):
    for i_batch, sample_batched in enumerate(dataset_loader):
        #print(i_batch, sample_batched['image'].size(),sample_batched['mask'].size())

        # Forward Propagation
        y_pred = roi.model(sample_batched['image'])
        # Compute and print loss
        loss = criterion(y_pred, sample_batched['mask'])
        print('epoch: ', epoch,' loss: ', loss.item())
                # Zero the gradients
        optimizer.zero_grad()

        # perform a backward pass (backpropagation)
        loss.backward()

        # Update the parameters
        optimizer.step()

In [ ]:
from PIL import Image
import numpy as np


for i_batch, sample_batched in enumerate(dataset_loader):
            #print(i_batch, sample_batched['image'].size(),sample_batched['mask'].size())

            # Forward Propagation
    y_pred = roi.model(sample_batched['image'])
            
    y_pred = np.reshape(y_pred.detach().numpy()[0], (32,32))
    img = Image.fromarray(y_pred, 'L')
    img.show()
    if i_batch == 2:
        break